In [2]:
import nbformat
from pathlib import Path

In [3]:
GOLD_DIR = "./gold"
p = Path(GOLD_DIR)

In [16]:
for nbfile in p.glob("*.ipynb"):
    nb = nbformat.read(nbfile, as_version=4)
    chunks = []
    for cell in nb.cells:
        if cell.cell_type == "markdown":
            chunks.append(
                "\n".join(
                    f"# {line}"
                for line
                in cell.source.splitlines()
                )
            )
        else:
            chunks.append(cell.source)

    nbfile.with_suffix(".py").write_text("\n\n".join(chunks))

3031